<h1>Equal-Weight S&P 500 Index Fund</h1>
<h2>Introduction and Library Imports</h2>


<h2>Library Imports</h2>


In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter
import math

<h2>Importing Our List of Stocks</h2>


In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

<h2>Acquiring an API Token</h2>

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

<h2>Making our first API Call</h2>

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

<h2>Parsing our api call</h2>


In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

<h2>Adding our Stocks Data to a pandas dataframe</h2>

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)


In [7]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,139.472,2277704427498,N/A


<h2>Looping through the tickers in our list of stcocks</h2>

In [8]:
final_dataframe = pd.DataFrame(columns= my_columns)

for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()

    final_dataframe = final_dataframe.append(
        pd.Series([
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns),
    ignore_index = True
    )

<h2>Using Batch API calls to improve performance</h2>

In [9]:
def chunks(lst,n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [10]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_string = []
for i in range(0, len(symbol_groups)):
    symbol_string.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns= my_columns)

for symbol_string in symbol_string:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ], 
                index = my_columns
            ),ignore_index=True
        )

In [11]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,137.250,41994988248,N/A
1,AAL,23.690,15109234157,N/A
2,AAP,189.730,12359658756,N/A
3,AAPL,133.098,2255963139195,N/A
4,ABBV,109.900,197736614473,N/A
...,...,...,...,...
500,YUM,116.950,34474752924,N/A
501,ZBH,170.120,35871934541,N/A
502,ZBRA,515.380,27753531887,N/A
503,ZION,56.664,9491594823,N/A


<h2>Calculatting the number of shares to buy</h2>


In [16]:
portfolio_size = input("Enter the value of your portfolio: ")

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number \n Please try again")

    portfolio_size = input("Enter the value of your portfolio: ")

    val = float(portfolio_size)

12.0
